In [1]:
%cd ~/REVIVAL2

/disk2/fli/REVIVAL2


In [2]:
import pandas as pd
from scipy.stats import spearmanr

In [3]:
df = pd.read_csv("/disk2/fli/REVIVAL2/zs/hydro/af3/struct_joint/ParLQ.csv")
df.head()

,var,fitness,pocket-plip-sasa_0,pocket-plip-sasa_1,pocket-plip-sasa_2,pocket-plip-sasa_3,pocket-plip-sasa_4,pocket-plip-sasa_avg,pocket-plip-kd_0,pocket-plip-kd_1,...,pocket-subcofcentroid-sasa-theor - substrate-sasa,pocket-subcofcentroid-sasa-theor - substrate+cofactor-sasa,pocket-subcofcentroid-sasa-emp - substrate-logp,pocket-subcofcentroid-sasa-emp - substrate-tpsa,pocket-subcofcentroid-sasa-emp - substrate-asa,pocket-subcofcentroid-sasa-emp - substrate+cofactor-logp,pocket-subcofcentroid-sasa-emp - substrate+cofactor-tpsa,pocket-subcofcentroid-sasa-emp - substrate+cofactor-asa,pocket-subcofcentroid-sasa-emp - substrate-sasa,pocket-subcofcentroid-sasa-emp - substrate+cofactor-sasa
0,F89A,0.000000,5538.606031,5448.358696,5652.553339,5548.180498,5448.575258,5527.254764,0.135897,0.144872,...,-73.391471,-906.827169,208.819695,201.927895,136.158018,206.849555,73.707895,-196.501676,-83.707261,-917.142958
1,F89C,4.215013,5531.955778,5450.651367,5609.420158,5540.730129,5443.174849,5515.186456,0.144872,0.153846,...,-81.199735,-889.878690,210.240747,203.348947,137.579071,208.270607,75.128947,-195.080623,-92.094471,-900.773427
2,F89D,1.585337,5566.070851,5492.035568,5617.457486,5476.191911,5461.263423,5522.603848,0.124051,0.124051,...,-76.553885,-880.211674,212.293379,205.401579,139.631702,210.323239,77.181579,-193.027992,-86.764411,-890.422200
3,F89E,2.487884,5557.648045,5438.427160,5610.336314,5495.966048,5535.882304,5527.651974,0.124051,0.076923,...,-69.098433,-892.339900,213.714432,206.822632,141.052755,211.744292,78.602632,-191.606939,-79.466854,-902.708321
4,F89G,0.000000,5286.482508,5504.399128,5582.748381,5505.503320,5474.431107,5470.712889,0.294595,0.035443,...,-82.185429,-892.183790,207.556537,200.664737,134.894860,205.586397,72.444737,-197.764834,-92.448587,-902.446948


In [4]:
for c in df.columns:
    if "substrate-" in c:
        print(c)

substrate-logp
substrate-tpsa
substrate-asa
substrate-sasa
pocket-plip-sasa - substrate-logp
pocket-plip-sasa - substrate-tpsa
pocket-plip-sasa - substrate-asa
pocket-plip-sasa - substrate-sasa
pocket-subcentroid-sasa - substrate-logp
pocket-subcentroid-sasa - substrate-tpsa
pocket-subcentroid-sasa - substrate-asa
pocket-subcentroid-sasa - substrate-sasa
pocket-subcofcentroid-sasa - substrate-logp
pocket-subcofcentroid-sasa - substrate-tpsa
pocket-subcofcentroid-sasa - substrate-asa
pocket-subcofcentroid-sasa - substrate-sasa
pocket-plip-kd - substrate-logp
pocket-plip-kd - substrate-tpsa
pocket-plip-kd - substrate-asa
pocket-plip-kd - substrate-sasa
pocket-plip-hw - substrate-logp
pocket-plip-hw - substrate-tpsa
pocket-plip-hw - substrate-asa
pocket-plip-hw - substrate-sasa
pocket-plip-ec - substrate-logp
pocket-plip-ec - substrate-tpsa
pocket-plip-ec - substrate-asa
pocket-plip-ec - substrate-sasa
pocket-plip-sasa-theor - substrate-logp
pocket-plip-sasa-theor - substrate-tpsa
pocket-

In [6]:
for c in df.columns:
    if " - " in c:
        print(c)

pocket-plip-sasa - substrate-logp
pocket-plip-sasa - substrate-tpsa
pocket-plip-sasa - substrate-asa
pocket-plip-sasa - substrate+cofactor-logp
pocket-plip-sasa - substrate+cofactor-tpsa
pocket-plip-sasa - substrate+cofactor-asa
pocket-plip-sasa - substrate-sasa
pocket-plip-sasa - substrate+cofactor-sasa
pocket-subcentroid-sasa - substrate-logp
pocket-subcentroid-sasa - substrate-tpsa
pocket-subcentroid-sasa - substrate-asa
pocket-subcentroid-sasa - substrate+cofactor-logp
pocket-subcentroid-sasa - substrate+cofactor-tpsa
pocket-subcentroid-sasa - substrate+cofactor-asa
pocket-subcentroid-sasa - substrate-sasa
pocket-subcentroid-sasa - substrate+cofactor-sasa
pocket-subcofcentroid-sasa - substrate-logp
pocket-subcofcentroid-sasa - substrate-tpsa
pocket-subcofcentroid-sasa - substrate-asa
pocket-subcofcentroid-sasa - substrate+cofactor-logp
pocket-subcofcentroid-sasa - substrate+cofactor-tpsa
pocket-subcofcentroid-sasa - substrate+cofactor-asa
pocket-subcofcentroid-sasa - substrate-sasa

In [14]:
for c in df.columns:
    if "_avg" in c or "_agg" in c:
        print(c)

pocket-plip-sasa_avg
pocket-plip-kd_avg
pocket-plip-hw_avg
pocket-plip-ec_avg
pocket-subcentroid-sasa_avg
pocket-subcentroid-kd_avg
pocket-subcentroid-hw_avg
pocket-subcentroid-ec_avg
pocket-subcofcentroid-sasa_avg
pocket-subcofcentroid-kd_avg
pocket-subcofcentroid-hw_avg
pocket-subcofcentroid-ec_avg
pocket-plip-sasa_agg
pocket-subcentroid-sasa_agg
pocket-subcofcentroid-sasa_agg
pocket-plip-kd_agg
pocket-plip-hw_agg
pocket-plip-ec_agg
pocket-subcentroid-kd_agg
pocket-subcentroid-hw_agg
pocket-subcentroid-ec_agg
pocket-subcofcentroid-kd_agg
pocket-subcofcentroid-hw_agg
pocket-subcofcentroid-ec_agg


In [7]:
(
    spearmanr(df["fitness"], df["pocket-plip-sasa_avg"] - df["substrate-sasa"]),
    spearmanr(df["fitness"], df["pocket-plip-sasa_agg"] - df["substrate-sasa"]),
    spearmanr(df["fitness"], df["pocket-subcentroid-sasa_avg"] - df["substrate-sasa"]),
    spearmanr(df["fitness"], df["pocket-subcentroid-sasa_agg"] - df["substrate-sasa"]),
    spearmanr(df["fitness"], df["pocket-subcofcentroid-sasa_avg"] - df["substrate-sasa"]),
    spearmanr(df["fitness"], df["pocket-subcofcentroid-sasa_agg"] - df["substrate-sasa"])
    )

(SignificanceResult(statistic=0.04755046071580823, pvalue=0.2934973385004446),
 SignificanceResult(statistic=-0.0052103533875718705, pvalue=0.908411594801199),
 SignificanceResult(statistic=-0.14962075573458292, pvalue=0.0008929237610688365),
 SignificanceResult(statistic=-0.2227753106819802, pvalue=6.309875734327112e-07),
 SignificanceResult(statistic=-0.01026826957771358, pvalue=0.8206389153478179),
 SignificanceResult(statistic=0.051732411264243536, pvalue=0.25304420885109474))

In [25]:
(
    spearmanr(df["fitness"], df["pocket-plip-sasa_avg"] - df["substrate+cofactor-sasa"]),
    spearmanr(df["fitness"], df["pocket-plip-sasa_agg"] - df["substrate+cofactor-sasa"]),
    spearmanr(df["fitness"], df["pocket-subcentroid-sasa_avg"] - df["substrate+cofactor-sasa"]),
    spearmanr(df["fitness"], df["pocket-subcentroid-sasa_agg"] - df["substrate+cofactor-sasa"]),
    spearmanr(df["fitness"], df["pocket-subcofcentroid-sasa_avg"] - df["substrate+cofactor-sasa"]),
    spearmanr(df["fitness"], df["pocket-subcofcentroid-sasa_agg"] - df["substrate+cofactor-sasa"])
    )

(SignificanceResult(statistic=0.013542022865581376, pvalue=0.7649296996578994),
 SignificanceResult(statistic=-0.023500879102107675, pvalue=0.6037906054166439),
 SignificanceResult(statistic=-0.14254489540663037, pvalue=0.0015589739267190086),
 SignificanceResult(statistic=-0.21276984099665794, pvalue=2.010606097505555e-06),
 SignificanceResult(statistic=-0.0013998824463902538, pvalue=0.9753424387361698),
 SignificanceResult(statistic=0.04565532312580998, pvalue=0.3131816291006604))

In [22]:
(
    spearmanr(df["fitness"], df["pocket-plip-kd_avg"] - df["substrate-logp"]),
    spearmanr(df["fitness"], df["pocket-plip-kd_agg"] - df["substrate-logp"]),
    spearmanr(df["fitness"], df["pocket-subcentroid-kd_avg"] - df["substrate-logp"]),
    spearmanr(df["fitness"], df["pocket-subcentroid-kd_agg"] - df["substrate-logp"]),
    spearmanr(df["fitness"], df["pocket-subcofcentroid-kd_avg"] - df["substrate-logp"]),
    spearmanr(df["fitness"], df["pocket-subcofcentroid-kd_agg"] - df["substrate-logp"])
    )

(SignificanceResult(statistic=-0.05109573479294273, pvalue=0.2589395615694466),
 SignificanceResult(statistic=-0.20667897726838366, pvalue=3.964184245256316e-06),
 SignificanceResult(statistic=-0.07657402327536836, pvalue=0.09041819792036414),
 SignificanceResult(statistic=-0.1411515719006619, pvalue=0.0017347861489189207),
 SignificanceResult(statistic=-0.132398038311777, pvalue=0.0033225328930425843),
 SignificanceResult(statistic=-0.16905294342660773, pvalue=0.00017012890748260384))

In [26]:
(
    spearmanr(df["fitness"], df["pocket-plip-kd_avg"] - df["substrate+cofactor-logp"]),
    spearmanr(df["fitness"], df["pocket-plip-kd_agg"] - df["substrate+cofactor-logp"]),
    spearmanr(df["fitness"], df["pocket-subcentroid-kd_avg"] - df["substrate+cofactor-logp"]),
    spearmanr(df["fitness"], df["pocket-subcentroid-kd_agg"] - df["substrate+cofactor-logp"]),
    spearmanr(df["fitness"], df["pocket-subcofcentroid-kd_avg"] - df["substrate+cofactor-logp"]),
    spearmanr(df["fitness"], df["pocket-subcofcentroid-kd_agg"] - df["substrate+cofactor-logp"])
    )

(SignificanceResult(statistic=-0.05109573479294273, pvalue=0.2589395615694466),
 SignificanceResult(statistic=-0.206624395365962, pvalue=3.988013623303961e-06),
 SignificanceResult(statistic=-0.07657402327536836, pvalue=0.09041819792036414),
 SignificanceResult(statistic=-0.14109325693924873, pvalue=0.0017425261222761526),
 SignificanceResult(statistic=-0.132398038311777, pvalue=0.0033225328930425843),
 SignificanceResult(statistic=-0.16906526137846298, pvalue=0.0001699399907565311))

In [31]:
import os
from MDAnalysis import Universe, AtomGroup
import freesasa
import tempfile

def apply_mutation(universe, mutation):
    """
    Apply a mutation to a protein structure.

    Args:
        universe (MDAnalysis.Universe): The MDAnalysis Universe object for the parent protein.
        mutation (tuple): Mutation specified as (resid, original_resname, new_resname).
                          Example: (56, "TRP", "ALA")

    Returns:
        MDAnalysis.Universe: Updated Universe with the mutation applied.
    """
    resid, original_resname, new_resname = mutation
    
    # Select the residue to mutate (ensure it's the Residue level)
    residue = universe.select_atoms(f"resid {resid} and resname {original_resname}").residues
    
    if len(residue) == 0:
        raise ValueError(f"Residue {original_resname} at {resid} not found in structure.")
    
    # Mutate the residue name
    residue.resnames = new_resname
    return universe

def calculate_variant_sasa(parent_pdb, mutations):
    """
    Calculate SASA for a protein variant based on the parent structure.

    Args:
        parent_pdb (str): Path to the parent protein's PDB file.
        mutations (list of tuples): List of mutations, each specified as (resid, original_resname, new_resname).

    Returns:
        float: Solvent-accessible surface area (SASA) for the variant.
    """
    # Load the parent structure
    universe = Universe(parent_pdb)

    # Apply all mutations
    for mutation in mutations:
        universe = apply_mutation(universe, mutation)

    # Write the mutated structure to a temporary PDB file
    temp_pdb = tempfile.NamedTemporaryFile(suffix=".pdb", delete=False).name
    universe.atoms.write(temp_pdb)  # Corrected file writing

    # Calculate SASA using FreeSASA
    structure = freesasa.Structure(temp_pdb)
    sasa = freesasa.calc(structure).totalArea()

    # Clean up
    os.remove(temp_pdb)

    return sasa


In [32]:
parent_pdb = "/disk2/fli/REVIVAL2/zs/plip/holo/ParLQ/ParLQ.pdb"
mutations = [
    (56, "TRP", "ALA"),  # Residue 56 mutated from ALA to GLY
    # (120, "TYR", "PHE"), # Residue 120 mutated from TYR to PHE
]

calculate_variant_sasa(parent_pdb, mutations)

FreeSASA: warning: atom 'ALA  CG ' unknown, guessing element is ' C', and radius 1.700 A
FreeSASA: warning: atom 'ALA  CD1' unknown, guessing element is ' C', and radius 1.700 A
FreeSASA: warning: atom 'ALA  CD2' unknown, guessing element is ' C', and radius 1.700 A
FreeSASA: warning: atom 'ALA  CE2' unknown, guessing element is ' C', and radius 1.700 A
FreeSASA: warning: atom 'ALA  CE3' unknown, guessing element is ' C', and radius 1.700 A
FreeSASA: warning: atom 'ALA  NE1' unknown, guessing element is ' N', and radius 1.550 A
FreeSASA: warning: atom 'ALA  CZ2' unknown, guessing element is ' C', and radius 1.700 A
FreeSASA: warning: atom 'ALA  CZ3' unknown, guessing element is ' C', and radius 1.700 A
FreeSASA: warning: atom 'ALA  CH2' unknown, guessing element is ' C', and radius 1.700 A


10003.434390192833

In [ ]:
# Given list of terms
import itertools
plipterms = [
    "substrate-logp_frompdb",
    "substrate-tpsa_frompdb",
    "substrate-asa_frompdb",
    "substrate+cofactor-logp_frompdb",
    "substrate+cofactor-tpsa_frompdb",
    "substrate+cofactor-asa_frompdb",
    "pocket-plip-sasa-emp_frompdb",
    "pocket-plip-sasa-theor_frompdb",
    "pocket-plip-sasa-mut_frompdb",
    "pocket-combo-sasa-emp_frompdb",
    "pocket-combo-sasa-theor_frompdb",
    "pocket-combo-sasa-mut_frompdb",
    "pocket-ligandcentroid-sasa-emp_frompdb",
    "pocket-ligandcentroid-sasa-theor_frompdb",
    "pocket-ligandcentroid-sasa-mut_frompdb",
    "pocket-plip-kd_frompdb",
    "pocket-plip-hw_frompdb",
    "pocket-plip-ec_frompdb",
    "pocket-combo-kd_frompdb",
    "pocket-combo-hw_frompdb",
    "pocket-combo-ec_frompdb",
    "pocket-ligandcentroid-kd_frompdb",
    "pocket-ligandcentroid-hw_frompdb",
    "pocket-ligandcentroid-ec_frompdb",
]

# Separate substrate terms and pocket terms
substrate_terms = [t for t in plipterms if t.startswith("substrate")]
pocket_terms = [t for t in plipterms if t.startswith("pocket")]

plip_df_frompdb = df_rho[plipterms]

for p, s in itertools.product(pocket_terms, substrate_terms):
    plip_df_frompdb[f"{p} - {s}"] = plip_df_frompdb[p] - plip_df_frompdb[s]


# Set the figure size
plt.figure(figsize=(18, 6))

# Create a heatmap
sns.heatmap(plip_df_frompdb, cmap="vlag", annot=True, fmt=".1f", linewidths=0.5)

# Show the plot
plt.title("Bond distance")
plt.show()